## Analisando dados em um notebook

In [18]:
import pandas as pd

In [85]:
zonas = pd.read_csv('dados/bairros_zonas.csv')
zonas.head()

,Zona,Bairro
0,4,BOTAFOGO
1,4,HUMAITA
2,4,FLAMENGO
3,4,URCA
4,5,COPACABA\rNA


In [26]:
resultados = pd.read_csv('dados/votacao_candidato_munzona_2016_RJ.csv', delimiter=';', encoding = "ISO-8859-1")

resultados.head(5)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_VOTO_EM_TRANSITO,QT_VOTOS_NOMINAIS
0,10/11/2019,00:53:10,2016,2,Eleição Ordinária,1,220,ELEIÇÕES MUNICIPAIS 2016,02/10/2016,M,...,36,PTC,Partido Trabalhista Cristão,190000000248,UNIÃO TRABALHISTA CRISTÃ,PT do B / PTC,5,SUPLENTE,N,0
1,10/11/2019,00:53:10,2016,2,Eleição Ordinária,1,220,ELEIÇÕES MUNICIPAIS 2016,02/10/2016,M,...,70,PT do B,Partido Trabalhista do Brasil,190000000248,UNIÃO TRABALHISTA CRISTÃ,PT do B / PTC,5,SUPLENTE,N,0
2,10/11/2019,00:53:10,2016,2,Eleição Ordinária,1,220,ELEIÇÕES MUNICIPAIS 2016,02/10/2016,M,...,70,PT do B,Partido Trabalhista do Brasil,190000000248,UNIÃO TRABALHISTA CRISTÃ,PT do B / PTC,5,SUPLENTE,N,0
3,10/11/2019,00:53:10,2016,2,Eleição Ordinária,1,220,ELEIÇÕES MUNICIPAIS 2016,02/10/2016,M,...,70,PT do B,Partido Trabalhista do Brasil,190000000248,UNIÃO TRABALHISTA CRISTÃ,PT do B / PTC,5,SUPLENTE,N,0
4,10/11/2019,00:53:10,2016,2,Eleição Ordinária,1,220,ELEIÇÕES MUNICIPAIS 2016,02/10/2016,M,...,70,PT do B,Partido Trabalhista do Brasil,190000000248,UNIÃO TRABALHISTA CRISTÃ,PT do B / PTC,5,SUPLENTE,N,2


In [27]:
resultados.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'SQ_CANDIDATO',
       'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO',
       'NM_SOCIAL_CANDIDATO', 'CD_SITUACAO_CANDIDATURA',
       'DS_SITUACAO_CANDIDATURA', 'CD_DETALHE_SITUACAO_CAND',
       'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO', 'NR_PARTIDO', 'SG_PARTIDO',
       'NM_PARTIDO', 'SQ_COLIGACAO', 'NM_COLIGACAO', 'DS_COMPOSICAO_COLIGACAO',
       'CD_SIT_TOT_TURNO', 'DS_SIT_TOT_TURNO', 'ST_VOTO_EM_TRANSITO',
       'QT_VOTOS_NOMINAIS'],
      dtype='object')

In [69]:
votacoes_rj = resultados.loc[(resultados['CD_MUNICIPIO'] == 60011) & 
               ((resultados['DS_SIT_TOT_TURNO'] == 'ELEITO') |
               (resultados['DS_SIT_TOT_TURNO'] == 'ELEITO POR MÉDIA') |
               (resultados['DS_SIT_TOT_TURNO'] == 'ELEITO POR QP'))][['ANO_ELEICAO', 'NR_TURNO', 
              'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'SQ_CANDIDATO', 
              'NM_CANDIDATO', 'NM_URNA_CANDIDATO', 'DS_SITUACAO_CANDIDATURA', 'SG_PARTIDO', 
              'NM_PARTIDO', 'QT_VOTOS_NOMINAIS', 'ST_VOTO_EM_TRANSITO', 
              'DS_SIT_TOT_TURNO']]

votacoes_rj.head()

,ANO_ELEICAO,NR_TURNO,NM_MUNICIPIO,NR_ZONA,CD_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,DS_SITUACAO_CANDIDATURA,SG_PARTIDO,NM_PARTIDO,QT_VOTOS_NOMINAIS,ST_VOTO_EM_TRANSITO,DS_SIT_TOT_TURNO
1304,2016,1,RIO DE JANEIRO,211,13,190000003862,ROGERIO DE CASTRO LOPES,ROGERIO ROCAL,APTO,PTB,Partido Trabalhista Brasileiro,4,N,ELEITO POR MÉDIA
1305,2016,1,RIO DE JANEIRO,21,13,190000010725,ADEJAIR SANCHES DE AGUIAR,JAIR DA MENDES GOMES,APTO,PMN,Partido da Mobilização Nacional,13,N,ELEITO POR MÉDIA
1306,2016,1,RIO DE JANEIRO,217,13,190000010725,ADEJAIR SANCHES DE AGUIAR,JAIR DA MENDES GOMES,APTO,PMN,Partido da Mobilização Nacional,577,N,ELEITO POR MÉDIA
1307,2016,1,RIO DE JANEIRO,165,13,190000010725,ADEJAIR SANCHES DE AGUIAR,JAIR DA MENDES GOMES,APTO,PMN,Partido da Mobilização Nacional,1,N,ELEITO POR MÉDIA
1308,2016,1,RIO DE JANEIRO,161,13,190000010725,ADEJAIR SANCHES DE AGUIAR,JAIR DA MENDES GOMES,APTO,PMN,Partido da Mobilização Nacional,17,N,ELEITO POR MÉDIA


In [71]:
total_votos = votacoes_rj.groupby('NM_CANDIDATO').agg({'QT_VOTOS_NOMINAIS': 'sum'}).reset_index()
total_votos = total_votos.rename(columns={"QT_VOTOS_NOMINAIS": "TOTAL_VOTOS"})
total_votos

,NM_CANDIDATO,TOTAL_VOTOS
0,ADALMIR D AGUILA DE SOUZA,8804
1,ADEJAIR SANCHES DE AGUIAR,8112
2,ALEXANDRE ISQUIERDO MOREIRA,24701
3,ANTONIO JOSE PAPERA DE AZEVEDO,21565
4,CARLO FERREIRA DE CAIADO CASTRO,28122
5,CARLOS EDUARDO DE MATTOS,19822
6,CARLOS NANTES BOLSONARO,106657
7,CELIO CESAR LUPPARELLI FARIA,8692
8,CESAR EPITÁCIO MAIA,71468
9,CLAUDIO BONFIM DE CASTRO E SILVA,10262


In [72]:
votacoes_rj = votacoes_rj.merge(total_votos, on='NM_CANDIDATO')
votacoes_rj

,ANO_ELEICAO,NR_TURNO,NM_MUNICIPIO,NR_ZONA,CD_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,DS_SITUACAO_CANDIDATURA,SG_PARTIDO,NM_PARTIDO,QT_VOTOS_NOMINAIS,ST_VOTO_EM_TRANSITO,DS_SIT_TOT_TURNO,TOTAL_VOTOS
0,2016,1,RIO DE JANEIRO,211,13,190000003862,ROGERIO DE CASTRO LOPES,ROGERIO ROCAL,APTO,PTB,Partido Trabalhista Brasileiro,4,N,ELEITO POR MÉDIA,15055
1,2016,1,RIO DE JANEIRO,19,13,190000003862,ROGERIO DE CASTRO LOPES,ROGERIO ROCAL,APTO,PTB,Partido Trabalhista Brasileiro,2,N,ELEITO POR MÉDIA,15055
2,2016,1,RIO DE JANEIRO,215,13,190000003862,ROGERIO DE CASTRO LOPES,ROGERIO ROCAL,APTO,PTB,Partido Trabalhista Brasileiro,6,N,ELEITO POR MÉDIA,15055
3,2016,1,RIO DE JANEIRO,168,13,190000003862,ROGERIO DE CASTRO LOPES,ROGERIO ROCAL,APTO,PTB,Partido Trabalhista Brasileiro,4,N,ELEITO POR MÉDIA,15055
4,2016,1,RIO DE JANEIRO,165,13,190000003862,ROGERIO DE CASTRO LOPES,ROGERIO ROCAL,APTO,PTB,Partido Trabalhista Brasileiro,8,N,ELEITO POR MÉDIA,15055
5,2016,1,RIO DE JANEIRO,24,13,190000003862,ROGERIO DE CASTRO LOPES,ROGERIO ROCAL,APTO,PTB,Partido Trabalhista Brasileiro,68,N,ELEITO POR MÉDIA,15055
6,2016,1,RIO DE JANEIRO,161,13,190000003862,ROGERIO DE CASTRO LOPES,ROGERIO ROCAL,APTO,PTB,Partido Trabalhista Brasileiro,4,N,ELEITO POR MÉDIA,15055
7,2016,1,RIO DE JANEIRO,237,13,190000003862,ROGERIO DE CASTRO LOPES,ROGERIO ROCAL,APTO,PTB,Partido Trabalhista Brasileiro,139,N,ELEITO POR MÉDIA,15055
8,2016,1,RIO DE JANEIRO,121,13,190000003862,ROGERIO DE CASTRO LOPES,ROGERIO ROCAL,APTO,PTB,Partido Trabalhista Brasileiro,0,N,ELEITO POR MÉDIA,15055
9,2016,1,RIO DE JANEIRO,123,13,190000003862,ROGERIO DE CASTRO LOPES,ROGERIO ROCAL,APTO,PTB,Partido Trabalhista Brasileiro,10,N,ELEITO POR MÉDIA,15055


In [76]:
votacoes_rj = votacoes_rj.sort_values(['NM_CANDIDATO', 'QT_VOTOS_NOMINAIS'], ascending=False).groupby('NM_CANDIDATO').head(5)
votacoes_rj

,ANO_ELEICAO,NR_TURNO,NM_MUNICIPIO,NR_ZONA,CD_CARGO,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,DS_SITUACAO_CANDIDATURA,SG_PARTIDO,NM_PARTIDO,QT_VOTOS_NOMINAIS,ST_VOTO_EM_TRANSITO,DS_SIT_TOT_TURNO,TOTAL_VOTOS
2427,2016,1,RIO DE JANEIRO,25,13,190000026194,WILLIAN CARVALHO DOS SANTOS,WILLIAN COELHO,APTO,PMDB,Partido do Movimento Democrático Brasileiro,11081,N,ELEITO POR QP,20678
2426,2016,1,RIO DE JANEIRO,241,13,190000026194,WILLIAN CARVALHO DOS SANTOS,WILLIAN COELHO,APTO,PMDB,Partido do Movimento Democrático Brasileiro,1697,N,ELEITO POR QP,20678
2510,2016,1,RIO DE JANEIRO,243,13,190000026194,WILLIAN CARVALHO DOS SANTOS,WILLIAN COELHO,APTO,PMDB,Partido do Movimento Democrático Brasileiro,1184,N,ELEITO POR QP,20678
2492,2016,1,RIO DE JANEIRO,240,13,190000026194,WILLIAN CARVALHO DOS SANTOS,WILLIAN COELHO,APTO,PMDB,Partido do Movimento Democrático Brasileiro,791,N,ELEITO POR QP,20678
2505,2016,1,RIO DE JANEIRO,125,13,190000026194,WILLIAN CARVALHO DOS SANTOS,WILLIAN COELHO,APTO,PMDB,Partido do Movimento Democrático Brasileiro,671,N,ELEITO POR QP,20678
285,2016,1,RIO DE JANEIRO,246,13,190000012843,VERÔNICA CHAVES DE CARVALHO COSTA,VERÔNICA COSTA,APTO,PMDB,Partido do Movimento Democrático Brasileiro,556,N,ELEITO POR MÉDIA,19946
209,2016,1,RIO DE JANEIRO,182,13,190000012843,VERÔNICA CHAVES DE CARVALHO COSTA,VERÔNICA COSTA,APTO,PMDB,Partido do Movimento Democrático Brasileiro,549,N,ELEITO POR MÉDIA,19946
281,2016,1,RIO DE JANEIRO,9,13,190000012843,VERÔNICA CHAVES DE CARVALHO COSTA,VERÔNICA COSTA,APTO,PMDB,Partido do Movimento Democrático Brasileiro,520,N,ELEITO POR MÉDIA,19946
210,2016,1,RIO DE JANEIRO,185,13,190000012843,VERÔNICA CHAVES DE CARVALHO COSTA,VERÔNICA COSTA,APTO,PMDB,Partido do Movimento Democrático Brasileiro,489,N,ELEITO POR MÉDIA,19946
262,2016,1,RIO DE JANEIRO,180,13,190000012843,VERÔNICA CHAVES DE CARVALHO COSTA,VERÔNICA COSTA,APTO,PMDB,Partido do Movimento Democrático Brasileiro,475,N,ELEITO POR MÉDIA,19946


In [78]:
votacoes_rj['PROPORCAO'] = votacoes_rj['QT_VOTOS_NOMINAIS']/votacoes_rj['TOTAL_VOTOS']
votacoes_rj

In [80]:
votacoes_rj['PORCENTAGEM'] = votacoes_rj['PROPORCAO']*100
votacoes_rj[['NM_CANDIDATO', 'QT_VOTOS_NOMINAIS', 'TOTAL_VOTOS', 'PROPORCAO']]

,QT_VOTOS_NOMINAIS,TOTAL_VOTOS,PROPORCAO
2427,11081,20678,0.535884
2426,1697,20678,0.082068
2510,1184,20678,0.057259
2492,791,20678,0.038253
2505,671,20678,0.032450
285,556,19946,0.027875
209,549,19946,0.027524
281,520,19946,0.026070
210,489,19946,0.024516
262,475,19946,0.023814


In [81]:
votacoes_rj[['NM_CANDIDATO', 'QT_VOTOS_NOMINAIS', 'TOTAL_VOTOS', 'PROPORCAO']]

,NM_CANDIDATO,QT_VOTOS_NOMINAIS,TOTAL_VOTOS,PROPORCAO
2427,WILLIAN CARVALHO DOS SANTOS,11081,20678,0.535884
2426,WILLIAN CARVALHO DOS SANTOS,1697,20678,0.082068
2510,WILLIAN CARVALHO DOS SANTOS,1184,20678,0.057259
2492,WILLIAN CARVALHO DOS SANTOS,791,20678,0.038253
2505,WILLIAN CARVALHO DOS SANTOS,671,20678,0.032450
285,VERÔNICA CHAVES DE CARVALHO COSTA,556,19946,0.027875
209,VERÔNICA CHAVES DE CARVALHO COSTA,549,19946,0.027524
281,VERÔNICA CHAVES DE CARVALHO COSTA,520,19946,0.026070
210,VERÔNICA CHAVES DE CARVALHO COSTA,489,19946,0.024516
262,VERÔNICA CHAVES DE CARVALHO COSTA,475,19946,0.023814


In [84]:
votacoes_rj.loc[votacoes_rj['NM_URNA_CANDIDATO'] == 'MARIELLE FRANCO'][['NR_ZONA',
                                                                        'NM_CANDIDATO', 
                                                                        'QT_VOTOS_NOMINAIS',
                                                                        'TOTAL_VOTOS', 
                                                                        'PROPORCAO',
                                                                        'PORCENTAGEM'
                                                                       ]]

,NR_ZONA,NM_CANDIDATO,QT_VOTOS_NOMINAIS,TOTAL_VOTOS,PROPORCAO,PORCENTAGEM
4464,16,MARIELLE FRANCISCO DA SILVA,2237,46502,0.048105,4.810546
4487,4,MARIELLE FRANCISCO DA SILVA,1926,46502,0.041418,4.141757
4501,7,MARIELLE FRANCISCO DA SILVA,1837,46502,0.039504,3.950368
4481,161,MARIELLE FRANCISCO DA SILVA,1688,46502,0.036300,3.629951
4514,9,MARIELLE FRANCISCO DA SILVA,1333,46502,0.028665,2.866543


In [90]:
zonas.loc[(zonas['Zona'] == 16) | 
          (zonas['Zona'] == 4) | 
          (zonas['Zona'] == 7) | 
          (zonas['Zona'] == 161) | 
          (zonas['Zona'] == 9)]

,Zona,Bairro
0,4,BOTAFOGO
1,4,HUMAITA
2,4,FLAMENGO
3,4,URCA
6,7,TIJUCA
15,9,BARRA DA\rTIJUCA
16,9,CAMORIM
17,9,VARGEM\rGRANDE
18,9,VARGEM\rPEQUENA
19,9,RECREIO


Resultados estão de acordo com a notícias publicada pela Folha de [São Paulo](https://www1.folha.uol.com.br/cotidiano/2018/03/marielle-franco-foi-mais-votada-em-bairros-nobres-do-rio-de-janeiro.shtml)

In [91]:
votacoes_rj.to_csv('dados/top_5_votacao.csv', index=False)